In [3]:
pip install torch torchvision matplotlib efficientnet_pytorch


  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached 

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet50
import matplotlib.pyplot as plt


In [8]:
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Loading and splitting the dataset
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

total_train = len(trainset)
val_size = int(0.1 * total_train)
train_size = total_train - val_size
train_dataset, val_dataset = torch.utils.data.random_split(trainset, [train_size, val_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [9]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.resnet = resnet50(pretrained=True)
        # Adjust for CIFAR-100 class count
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 100)

    def forward(self, x):
        return self.resnet(x)

model = ResNet()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 127MB/s]


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [11]:
def train(model, trainloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch, Batch]: [{epoch+1}, {i+1}], Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

def validate(model, valloader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100. * correct / total
    print(f'Validation Loss: {val_loss / len(valloader):.3f}, Accuracy: {accuracy:.2f}%')
    return val_loss / len(valloader), accuracy


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

early_stopping_patience = 5
early_stopping_counter = 0
min_val_loss = float('inf')

for epoch in range(10):  # Adjust the number of epochs as needed
    train(model, trainloader, criterion, optimizer, device)
    val_loss, _ = validate(model, valloader, criterion, device)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[Epoch, Batch]: [1, 100], Loss: 2.627
[Epoch, Batch]: [1, 200], Loss: 1.087
[Epoch, Batch]: [1, 300], Loss: 0.959
Validation Loss: 0.989, Accuracy: 71.54%
[Epoch, Batch]: [2, 100], Loss: 0.698
[Epoch, Batch]: [2, 200], Loss: 0.694
[Epoch, Batch]: [2, 300], Loss: 0.695
Validation Loss: 0.890, Accuracy: 74.74%
[Epoch, Batch]: [3, 100], Loss: 0.484
[Epoch, Batch]: [3, 200], Loss: 0.486
[Epoch, Batch]: [3, 300], Loss: 0.512
Validation Loss: 0.864, Accuracy: 75.64%
[Epoch, Batch]: [4, 100], Loss: 0.361
[Epoch, Batch]: [4, 200], Loss: 0.358
[Epoch, Batch]: [4, 300], Loss: 0.393
Validation Loss: 0.818, Accuracy: 77.62%
[Epoch, Batch]: [5, 100], Loss: 0.263
[Epoch, Batch]: [5, 200], Loss: 0.277
[Epoch, Batch]: [5, 300], Loss: 0.307
Validation Loss: 0.813, Accuracy: 78.56%
[Epoch, Batch]: [6, 100], Loss: 0.205
[Epoch, Batch]: [6, 200], Loss: 0.223
[Epoch, Batch]: [6, 300], Loss: 0.234
Validation Loss: 0.823, Accuracy: 78.02%
[Epoch, Batch]: [7, 100], Loss: 0.166
[Epoch, Batch]: [7, 200], Loss: 

In [14]:
for epoch in range(10):  # Adjust the number of epochs as needed
    train(model, trainloader, criterion, optimizer, device)
    val_loss, _ = validate(model, valloader, criterion, device)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

[Epoch, Batch]: [1, 100], Loss: 0.087
[Epoch, Batch]: [1, 200], Loss: 0.102
[Epoch, Batch]: [1, 300], Loss: 0.109
Validation Loss: 0.790, Accuracy: 80.04%
[Epoch, Batch]: [2, 100], Loss: 0.087
[Epoch, Batch]: [2, 200], Loss: 0.082
[Epoch, Batch]: [2, 300], Loss: 0.099
Validation Loss: 0.793, Accuracy: 80.82%
[Epoch, Batch]: [3, 100], Loss: 0.065
[Epoch, Batch]: [3, 200], Loss: 0.070
[Epoch, Batch]: [3, 300], Loss: 0.075
Validation Loss: 0.833, Accuracy: 80.34%
Early stopping triggered.


In [15]:
def evaluate():
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy of the network on the 10000 test images: %d %%' % accuracy)

evaluate()

Accuracy of the network on the 10000 test images: 81 %
